In [1]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [2]:
from wefe.query import Query
from wefe.metrics import WEAT, RNSB, RND, MAC, ECT
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets.datasets import load_weat, fetch_eds, fetch_debias_multiclass, fetch_debiaswe, load_bingliu
from wefe.utils import run_queries, plot_queries_results, create_ranking, plot_ranking, plot_ranking_correlations
from plotly.subplots import make_subplots
import gensim

In [3]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"

In [4]:
def get_google_news_embeddings(filename,binary=False):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=binary, no_header=True)
    return w2v_model

In [5]:
fasttext_wiki_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T14:31:16.491415', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [6]:
fasttext_wiki_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/wiki-news-300d-1M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (999994, 300) matrix of type float32 from ../../Data//word embeddings/FastText/wiki-news-300d-1M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T14:32:26.760435', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [7]:
fasttext_cc_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1999995, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T14:34:48.808680', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [8]:
fasttext_cc_subwords_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"FastText/crawl-300d-2M-subword.vec")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2000000, 300) matrix of type float32 from ../../Data//word embeddings/FastText/crawl-300d-2M-subword.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T14:37:09.001755', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [9]:
debiased_w2v_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"/debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin",binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ../../Data//word embeddings//debiased_word_embedding/GoogleNews-vectors-negative300-hard-debiased.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-19T14:37:21.818349', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-gli

In [10]:
# load the weat word sets
WEAT_wordsets = load_weat()
RND_wordsets = fetch_eds()
sentiments_wordsets = load_bingliu()
debias_multiclass_wordsets = fetch_debias_multiclass()

#### WEAT Effect Size: represents a normalized measure that quantifies how far apart the two distributions of association between targets and attributes are.

#### WEAT score:  Its objective is to quantify the strength of association of both pair of sets through a permutation test. The score being positive and higher than one indicates that word2vec exhibits a moderately strong relationship between men's names and careers and women's names and family.

#### running multiple queries and multiple word embeddings

In [11]:
fasttext_wiki_model = WordEmbeddingModel(fasttext_wiki_model, 'fasttext_wiki')
fasttext_wiki_subwords_model = WordEmbeddingModel(fasttext_wiki_subwords_model, 'fasttext_wiki_subwords')
fasttext_cc_model = WordEmbeddingModel(fasttext_cc_model, 'fasttext_cc')
fasttext_cc_subwords_model = WordEmbeddingModel(fasttext_cc_subwords_model, 'fasttext_cc_subwords')
debiased_w2v_model = WordEmbeddingModel(debiased_w2v_model, 'debiased_w2v')

In [12]:
models = [fasttext_wiki_model, fasttext_wiki_subwords_model, fasttext_cc_model, fasttext_cc_subwords_model, debiased_w2v_model]

## Query ranking

In [13]:
# Create gender queries
eth_1 = Query([RND_wordsets['names_white'], RND_wordsets['names_black']],
              [WEAT_wordsets['pleasant_5'], WEAT_wordsets['unpleasant_5']],
              ['White last names', 'Black last names'],
              ['Pleasant', 'Unpleasant'])

eth_2 = Query([RND_wordsets['names_white'], RND_wordsets['names_asian']],
              [WEAT_wordsets['pleasant_5'], WEAT_wordsets['unpleasant_5']],
              ['White last names', 'Asian last names'],
              ['Pleasant', 'Unpleasant'])

eth_3 = Query([RND_wordsets['names_white'], RND_wordsets['names_hispanic']],
              [WEAT_wordsets['pleasant_5'], WEAT_wordsets['unpleasant_5']],
              ['White last names', 'Hispanic last names'],
              ['Pleasant', 'Unpleasant'])



eth_5 = Query(
    [RND_wordsets['names_white'], RND_wordsets['names_black']],
    [list(RND_wordsets['occupations_white']), list(RND_wordsets['occupations_black'])],
    ['White last names', 'Black last names'],
    ['Occupations white', 'Occupations black'])

eth_6 = Query(
    [RND_wordsets['names_white'], RND_wordsets['names_asian']],
    [list(RND_wordsets['occupations_white']), list(RND_wordsets['occupations_asian'])],
    ['White last names', 'Asian last names'],
    ['Occupations white', 'Occupations asian'])

eth_7 = Query(
    [RND_wordsets['names_white'], RND_wordsets['names_hispanic']],
    [list(RND_wordsets['occupations_white']), list(RND_wordsets['occupations_hispanic'])],
    ['White last names', 'Hispanic last names'],
    ['Occupations white', 'Occupations hispanic'])

eth_sent_1 = Query([RND_wordsets['names_white'], RND_wordsets['names_black']],
                   [
                       sentiments_wordsets['positive_words'],
                       sentiments_wordsets['negative_words']
                   ], ['White last names', 'Black last names'],
                   ['Positive words', 'Negative words'])

eth_sent_2 = Query([RND_wordsets['names_white'], RND_wordsets['names_asian']],
                   [
                       sentiments_wordsets['positive_words'],
                       sentiments_wordsets['negative_words']
                   ], ['White last names', 'Asian last names'],
                   ['Positive words', 'Negative words'])

eth_sent_3 = Query(
    [RND_wordsets['names_white'], RND_wordsets['names_hispanic']], [
        sentiments_wordsets['positive_words'],
        sentiments_wordsets['negative_words']
    ], ['White last names', 'Hispanic last names'],
    ['Positive words', 'Negative words'])



ethnicity_queries = [
    eth_1, eth_2, eth_3, eth_5, eth_6, eth_7, eth_sent_1, eth_sent_2,
    eth_sent_3]

In [14]:
# Run the queries WEAT
WEAT_ethnicity_results = run_queries(
    WEAT,
    ethnicity_queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    return_only_aggregation=True,
    lost_vocabulary_threshold = 0.5,
    queries_set_name='ethnicity Queries')


In [15]:
# Run the queries using RNSB
RNSB_ethnicity_results = run_queries(RNSB,
                                ethnicity_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                     #lost_vocabulary_threshold = 0.5,
                                return_only_aggregation=True,
                                queries_set_name='ethnicity Queries')


/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  probabilities = np.array([
/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  negative_probabilities = np.array(
/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. I

In [16]:
# Run the queries using RND
RND_ethnicity_results = run_queries(RND,
                                ethnicity_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                    lost_vocabulary_threshold = 0.5,
                                generate_subqueries=True,
                                    return_only_aggregation=True,
                                queries_set_name='ethnicity Queries')




In [17]:
# Run the queries using ECT
ECT_ethnicity_results = run_queries(ECT,
                                ethnicity_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                    return_only_aggregation=True,
                                generate_subqueries=True,
                                    lost_vocabulary_threshold = 0.5,
                                queries_set_name='ethnicity Queries')

In [18]:
WEAT_ethnicity_results

,WEAT: ethnicity Queries average of abs values score
model_name,
fasttext_wiki,0.110552
fasttext_wiki_subwords,0.162086
fasttext_cc,0.084091
fasttext_cc_subwords,0.082642
debiased_w2v,0.171114


In [19]:
RNSB_ethnicity_results

,RNSB: ethnicity Queries average of abs values score
model_name,
fasttext_wiki,0.088127
fasttext_wiki_subwords,0.008409
fasttext_cc,0.113996
fasttext_cc_subwords,0.023930
debiased_w2v,0.038671


In [20]:
RND_ethnicity_results

,RND: ethnicity Queries average of abs values score
model_name,
fasttext_wiki,0.086961
fasttext_wiki_subwords,0.045902
fasttext_cc,0.238755
fasttext_cc_subwords,0.088536
debiased_w2v,0.027600


In [21]:
ECT_ethnicity_results

,ECT: ethnicity Queries average of abs values score
model_name,
fasttext_wiki,0.835574
fasttext_wiki_subwords,0.837477
fasttext_cc,0.805789
fasttext_cc_subwords,0.828197
debiased_w2v,0.797371
